In [ ]:
# coding: utf-8
# 从压缩包中提取数据到csv文件
import os
import tarfile
import gzip
import zipfile
import csv
import re

tenhou_files_path = '../dataset/Tenhou'
files = os.listdir(tenhou_files_path)
pattern = re.compile(r'(\d{4})/scc')

for file in files:

    # 判断是否为压缩文件
    if not file.endswith('.zip'):
        continue

    with zipfile.ZipFile(os.path.join(tenhou_files_path, file), 'r') as zip_ref:

        namelist = zip_ref.namelist()
        extracted_data_filename = file[5:9] + ".csv"
        with open(os.path.join(tenhou_files_path, extracted_data_filename), 'w') as extracted_data_file:
            writer = csv.writer(extracted_data_file)
            writer.writerow(['date', 'time', 'game_rule', 'log_url'])

            for name in namelist:
                if name.endswith('.html.gz') and re.match(pattern, name):
                    year = name[-16:-12]
                    month = name[-12:-10]
                    day = name[-10:-8]
                    date = year + "/" + month + "/"+ day
                    print(date)

                    zip_ref.extract(name, os.path.join(tenhou_files_path, 'temp'))
                    with gzip.open(os.path.join(tenhou_files_path, 'temp', name), 'r') as html_file:
                        data_b = html_file.readlines()
                        data = [line.decode('utf-8').strip() for line in data_b]
                        # print(data[0])
                        # print(len(data))
                        # break
                        for i in range(len(data)):
                            split_data = data[i].split('|')
                            time = split_data[0].strip()
                            game_rule = split_data[2].strip()
                            raw_url = split_data[3].strip()
                            # print(split_data)
                            # break

                            try:
                                # 提取URL链接部分
                                start_index = raw_url.find('http')  # 找到URL的起始位置
                                end_index = raw_url.find('">')  # 找到URL的结束位置

                                if start_index != -1 and end_index != -1:
                                    url_part = raw_url[start_index:end_index]  # 提取URL部分

                                    # 替换"?log="为"log/?"
                                    modified_url = url_part.replace('?log=', 'log/?')

                                    # print(modified_url)  # 输出修改后的URL
                            
                            except:
                                modified_url = 'None'
                            
                            series = [date, time, game_rule, modified_url]
                            writer.writerow(series)


                if name.endswith('.html') and name.startswith(''):

                    year = name[-13:-9]
                    month = name[-9:-7]
                    day = name[-7:-5]
                    date = year + "/" + month + "/"+ day

                    with zip_ref.open(name, 'r') as html_file:
                        data_b = html_file.readlines()
                        data = [line.decode('utf-8').strip() for line in data_b]
                        # print(data[0])
                        # print(len(data))
                        # break
                        for i in range(len(data)):
                            split_data = data[i].split('|')
                            time = split_data[0].strip()
                            game_rule = split_data[2].strip()
                            raw_url = split_data[3].strip()
                            # print(split_data)
                            # break

                            try:
                                # 提取URL链接部分
                                start_index = raw_url.find('http')  # 找到URL的起始位置
                                end_index = raw_url.find('">')  # 找到URL的结束位置

                                if start_index != -1 and end_index != -1:
                                    url_part = raw_url[start_index:end_index]  # 提取URL部分

                                    # 替换"?log="为"log/?"
                                    modified_url = url_part.replace('?log=', 'log/?')

                                    # print(modified_url)  # 输出修改后的URL
                            
                            except:
                                modified_url = 'None'
                            
                            series = [date, time, game_rule, modified_url]
                            writer.writerow(series)

In [ ]:
# 筛选四鳳南喰赤的数据

import pandas as pd
import numpy as np
import os
import re



# 读取csv文件
tenhou_files_path = '../dataset/Tenhou'
files = os.listdir(tenhou_files_path)
pattern = re.compile(r'(\d{4})\.csv')

for file in files:
    if pattern.match(file):
        data = pd.read_csv(os.path.join(tenhou_files_path, file))
        nan_4_kuitan_aka = data[data['game_rule'].str.startswith('四鳳南喰赤')]
        print(file[:-4],"年总对局：")
        print(data.shape)
        print("四鳳南喰赤总对局：")
        print(nan_4_kuitan_aka.shape)
        nan_4_kuitan_aka.to_csv(os.path.join(tenhou_files_path, file[:-4] + '_nan_4_kuitan_aka.csv'), index=False)

In [ ]:
# 根据URL下载牌谱数据并保存

import urllib.request
import os
import pandas as pd
import gzip
import time

years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
csv_path = '../../dataset/Tenhou'
HEADER = {
    'Host': 'e.mjv.jp',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive'
}
for year in years:
    data = pd.read_csv(os.path.join(csv_path, f'{year}_nan_4_kuitan_aka.csv'))
    save_path = os.path.join(csv_path, f'{year}_nan_4_kuitan_aka')

    # 读取第一条数据
    for index, url in data['log_url'].items():
        req = urllib.request.Request(url, headers=HEADER)
        opener = urllib.request.build_opener()
        response = opener.open(req)
        if response.info().get('Content-Encoding') == 'gzip':
            file_name = str(index + 2) + '.gz'
            with open(os.path.join(save_path, file_name), 'wb') as f:
                f.write(response.read())



In [ ]:
log_contents = []
HEADER = {
    'Host': 'e.mjv.jp',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive'
}

data = pd.read_csv(f'../../dataset/Tenhou/{year[0]}_nan_4_kuitan_aka.csv')
urls = data['log_url'][0:10]

for index, url in urls.items():
    req = urllib.request.Request(url, headers=HEADER)
    opener = urllib.request.build_opener()
    response = opener.open(req)
    if response.info().get('Content-Encoding') == 'gzip':
        file_name = str(index + 2) + '.gz'
        with open(file_name, 'wb') as f:
            f.write(response.read())
